# Test database queries

**Goal of this notebook:** The original implementation of the `FieldManager.get_fields()` method in `db.py` was unbarebly slow. Here I am testing why and how to fix it.

In [3]:
from pandas import DataFrame
import timeit

from db import SQLiteConnection

In [4]:
db_file = 'test_strategy.sqlite3'

## Query `Fields` table

In [5]:
def query():
    with SQLiteConnection(db_file) as connection:
        cursor = connection.cursor()
        query = """\
                SELECT *
                FROM Fields
        """
        results = cursor.execute(query).fetchall()
    return results

In [6]:
DataFrame(query())

,field_id,fov,center_ra,center_dec,tilt,telescope_id,active
0,1,0.008727,0.000000,0.004363,0.0,1,1
1,2,0.008727,0.008678,0.004363,0.0,1,1
2,3,0.008727,0.017357,0.004363,0.0,1,1
3,4,0.008727,0.026035,0.004363,0.0,1,1
4,5,0.008727,0.034714,0.004363,0.0,1,1
...,...,...,...,...,...,...,...
145310,145311,0.010181,6.232514,-0.004000,0.0,2,1
145311,145312,0.010181,6.242649,-0.004000,0.0,2,1
145312,145313,0.010181,6.252783,-0.004000,0.0,2,1
145313,145314,0.010181,6.262917,-0.004000,0.0,2,1


Run time:

In [7]:
rep = 10
runtime = timeit.timeit(query, number=rep) / rep
print(f'Runtime: {runtime:.3f} s')

Runtime: 0.467 s


## Query `FieldsObs` view

In [19]:
def query():
    with SQLiteConnection(db_file) as connection:
        cursor = connection.cursor()
        query = """\
                SELECT *
                FROM FieldsObs
        """
        results = cursor.execute(query).fetchall()
    return results

In [20]:
DataFrame(query())

,field_id,fov,center_ra,center_dec,tilt,telescope,active,jd_first,jd_next,nobs_tot,nobs_done,nobs_pending
0,1,0.008727,0.000000,0.004363,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1
1,2,0.008727,0.008678,0.004363,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1
2,3,0.008727,0.017357,0.004363,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1
3,4,0.008727,0.026035,0.004363,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1
4,5,0.008727,0.034714,0.004363,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
145310,145311,0.010181,6.232514,-0.004000,0.0,SAAO,1,2460310.5,2460370.5,1,0,1
145311,145312,0.010181,6.242649,-0.004000,0.0,SAAO,1,2460310.5,2460370.5,1,0,1
145312,145313,0.010181,6.252783,-0.004000,0.0,SAAO,1,2460310.5,2460370.5,1,0,1
145313,145314,0.010181,6.262917,-0.004000,0.0,SAAO,1,2460310.5,2460370.5,1,0,1


Run time:

In [21]:
rep = 20
runtime = timeit.timeit(query, number=rep) / rep
print(f'Runtime: {runtime:.3f} s')

Runtime: 0.971 s


### Conditional search for fields with pending observations

In [22]:
def query():
    with SQLiteConnection(db_file) as connection:
        cursor = connection.cursor()
        query = """\
                SELECT *
                FROM FieldsObs
                WHERE nobs_pending > 0
        """
        results = cursor.execute(query).fetchall()
    return results

In [23]:
DataFrame(query())

,field_id,fov,center_ra,center_dec,tilt,telescope,active,jd_first,jd_next,nobs_tot,nobs_done,nobs_pending
0,1,0.008727,0.000000,0.004363,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1
1,2,0.008727,0.008678,0.004363,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1
2,3,0.008727,0.017357,0.004363,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1
3,4,0.008727,0.026035,0.004363,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1
4,5,0.008727,0.034714,0.004363,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
145310,145311,0.010181,6.232514,-0.004000,0.0,SAAO,1,2460310.5,2460370.5,1,0,1
145311,145312,0.010181,6.242649,-0.004000,0.0,SAAO,1,2460310.5,2460370.5,1,0,1
145312,145313,0.010181,6.252783,-0.004000,0.0,SAAO,1,2460310.5,2460370.5,1,0,1
145313,145314,0.010181,6.262917,-0.004000,0.0,SAAO,1,2460310.5,2460370.5,1,0,1


Run time:

In [24]:
rep = 20
runtime = timeit.timeit(query, number=rep) / rep
print(f'Runtime: {runtime:.3f} s')

Runtime: 0.920 s


## Query `Observable` view

**Note:** We cannot query the entire table which contains millions of rows. We perform a typical query for fields observable during a specific night:

In [7]:
night_start = '2024-01-15 23:00:00'
night_stop = '2024-01-16 04:00:00'

def query():
    with SQLiteConnection(db_file) as connection:
        cursor = connection.cursor()
        query = """\
                SELECT *
                FROM Observable
                WHERE (
                    date_start < '{0}'
                    AND date_stop > '{1}'
                    )
        """.format(night_stop, night_start)
        results = cursor.execute(query).fetchall()
    return results

In [8]:
DataFrame(query())

,field_id,date_start,date_stop,duration,status,setting_duration
0,2094,2024-01-16 01:06:00.000,2024-01-16 04:29:36.381,0.141393,rising,NaN
1,2095,2024-01-16 01:08:00.000,2024-01-16 04:29:36.381,0.140004,rising,NaN
2,2096,2024-01-16 01:10:00.000,2024-01-16 04:29:36.381,0.138616,rising,NaN
3,2097,2024-01-16 01:12:00.000,2024-01-16 04:29:36.381,0.137227,rising,NaN
4,2098,2024-01-16 01:14:00.000,2024-01-16 04:29:36.381,0.135838,rising,NaN
...,...,...,...,...,...,...
80796,145184,2024-01-16 02:33:00.000,2024-01-16 02:44:07.656,0.007728,rising,NaN
80797,145185,2024-01-16 02:35:00.000,2024-01-16 02:44:07.656,0.006339,rising,NaN
80798,145186,2024-01-16 02:38:00.000,2024-01-16 02:44:07.656,0.004255,rising,NaN
80799,145188,2024-01-16 02:42:00.000,2024-01-16 02:50:00.000,0.005556,rising,NaN


Run time:

In [9]:
rep = 10
runtime = timeit.timeit(query, number=rep) / rep
print(f'Runtime: {runtime:.3f} s')

Runtime: 2.469 s


## Joint query of `FieldsObs` and `Observable` view

**Note:** We cannot query the entire table which contains millions of rows. We perform a query for fields observable during a specific night, which is the typical usecase.

### No additional conditions

In [27]:
night_start = '2024-01-15 23:00:00'
night_stop = '2024-01-16 04:00:00'

def query():
    with SQLiteConnection(db_file) as connection:
        cursor = connection.cursor()
        query = """\
                SELECT *
                FROM FieldsObs AS f
                INNER JOIN Observable AS o
                ON f.field_id = o.field_id
                WHERE (
                    o.date_start < '{0}'
                    AND o.date_stop > '{1}'
                    )
        """.format(night_stop, night_start)
        results = cursor.execute(query).fetchall()
    return results

In [28]:
DataFrame(query())

,field_id,fov,center_ra,center_dec,tilt,telescope,active,jd_first,jd_next,nobs_tot,nobs_done,nobs_pending,date_start,date_stop,duration,status,setting_duration
0,2094,0.008727,3.653620,0.021817,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1,2024-01-16 01:06:00.000,2024-01-16 04:29:36.381,0.141393,rising,NaN
1,2095,0.008727,3.662299,0.021817,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1,2024-01-16 01:08:00.000,2024-01-16 04:29:36.381,0.140004,rising,NaN
2,2096,0.008727,3.670977,0.021817,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1,2024-01-16 01:10:00.000,2024-01-16 04:29:36.381,0.138616,rising,NaN
3,2097,0.008727,3.679655,0.021817,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1,2024-01-16 01:12:00.000,2024-01-16 04:29:36.381,0.137227,rising,NaN
4,2098,0.008727,3.688334,0.021817,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1,2024-01-16 01:14:00.000,2024-01-16 04:29:36.381,0.135838,rising,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80796,145184,0.010181,3.769911,-0.004000,0.0,SAAO,1,2460310.5,2460370.5,1,0,1,2024-01-16 02:33:00.000,2024-01-16 02:44:07.656,0.007728,rising,NaN
80797,145185,0.010181,3.780045,-0.004000,0.0,SAAO,1,2460310.5,2460370.5,1,0,1,2024-01-16 02:35:00.000,2024-01-16 02:44:07.656,0.006339,rising,NaN
80798,145186,0.010181,3.790180,-0.004000,0.0,SAAO,1,2460310.5,2460370.5,1,0,1,2024-01-16 02:38:00.000,2024-01-16 02:44:07.656,0.004255,rising,NaN
80799,145188,0.010181,3.810448,-0.004000,0.0,SAAO,1,2460310.5,2460370.5,1,0,1,2024-01-16 02:42:00.000,2024-01-16 02:50:00.000,0.005556,rising,NaN


Run time:

In [29]:
rep = 10
runtime = timeit.timeit(query, number=rep) / rep
print(f'Runtime: {runtime:.3f} s')

Runtime: 2.503 s


### Query only active fields associated with specific telescope

**Note:** This takes very long, when we try to fetch all results. For a quicker test, we only fetch the first row.

In [69]:
night_start = '2024-01-15 23:00:00'
night_stop = '2024-01-16 04:00:00'
telescope = 'Skinakas'

def query():
    with SQLiteConnection(db_file) as connection:
        cursor = connection.cursor()
        query = """\
                SELECT *
                FROM FieldsObs AS f
                INNER JOIN Observable AS o
                ON f.field_id = o.field_id
                WHERE (
                    o.date_start < '{0}'
                    AND o.date_stop > '{1}'
                    AND f.telescope = '{2}'
                    AND f.active = 1
                    )
        """.format(night_stop, night_start, telescope)
        results = cursor.execute(query).fetchone()
    return results

In [71]:
rep = 10
runtime = timeit.timeit(query, number=rep) / rep
print(f'Runtime: {runtime:.3f} s')

Runtime: 0.454 s


### Query only active fields with pending observations and associated with specific telescope

**Note:** This takes very long, when we try to fetch all results. For a quicker test, we only fetch the first row.

In [72]:
night_start = '2024-01-15 23:00:00'
night_stop = '2024-01-16 04:00:00'
telescope = 'Skinakas'

def query():
    with SQLiteConnection(db_file) as connection:
        cursor = connection.cursor()
        query = """\
                SELECT *
                FROM FieldsObs AS f
                INNER JOIN Observable AS o
                ON f.field_id = o.field_id
                WHERE (
                    o.date_start < '{0}'
                    AND o.date_stop > '{1}'
                    AND telescope = '{2}'
                    AND f.active = 1
                    AND f.nobs_pending > 0
                    )
        """.format(night_stop, night_start, telescope)
        results = cursor.execute(query).fetchone()
    return results

In [73]:
rep = 5
runtime = timeit.timeit(query, number=rep) / rep
print(f'Runtime: {runtime:.3f} s')

Runtime: 13.020 s


#### Conditional FieldsObs before join

**Note:** This takes very long, when we try to fetch all results. For a quicker test and comparability to the test above, we only fetch the first row.

In [74]:
night_start = '2024-01-15 23:00:00'
night_stop = '2024-01-16 04:00:00'
telescope = 'Skinakas'

def query():
    with SQLiteConnection(db_file) as connection:
        cursor = connection.cursor()
        query = """\
                SELECT *
                FROM (
                    SELECT *
                    FROM FieldsObs
                    WHERE (
                        active = 1
                        AND telescope = '{2}'
                        AND nobs_pending > 0
                        )
                    ) AS f
                INNER JOIN Observable AS o
                ON f.field_id = o.field_id
                WHERE (
                    o.date_start < '{0}'
                    AND o.date_stop > '{1}'
                    )
        """.format(night_stop, night_start, telescope)
        results = cursor.execute(query).fetchone()
    return results

In [75]:
rep = 5
runtime = timeit.timeit(query, number=rep) / rep
print(f'Runtime: {runtime:.3f} s')

Runtime: 12.171 s


**Result:** Does not provide a substantial runtime improvement.

#### Conditional Observable before join

**Note:** This takes very long, when we try to fetch all results. For a quicker test and comparability to the test above, we only fetch the first row.

In [81]:
night_start = '2024-01-15 23:00:00'
night_stop = '2024-01-16 04:00:00'
telescope = 'Skinakas'

def query():
    with SQLiteConnection(db_file) as connection:
        cursor = connection.cursor()
        query = """\
                SELECT *
                FROM FieldsObs AS f
                INNER JOIN (
                    SELECT *
                    FROM Observable 
                    WHERE (
                        date_start < '{0}'
                        AND date_stop > '{1}'
                        )
                    ) AS o
                ON f.field_id = o.field_id
                WHERE (
                    f.telescope = '{2}'
                    AND f.active = 1
                    AND f.nobs_pending > 0
                    )
        """.format(night_stop, night_start, telescope)
        results = cursor.execute(query).fetchone()
    return results

In [82]:
rep = 5
runtime = timeit.timeit(query, number=rep) / rep
print(f'Runtime: {runtime:.3f} s')

Runtime: 12.440 s


**Result:** Does not provide a substantial runtime improvement.

### Apply additional conditions in python

In [78]:
night_start = '2024-01-15 23:00:00'
night_stop = '2024-01-16 04:00:00'
telescope = 'Skinakas'
observed = None
pending = True
active = True

def query():
    with SQLiteConnection(db_file) as connection:
        cursor = connection.cursor()
        query = """\
                SELECT *
                FROM FieldsObs AS f
                INNER JOIN Observable AS o
                ON f.field_id = o.field_id
                WHERE (
                    o.date_start < '{0}'
                    AND o.date_stop > '{1}'
                    )
        """.format(night_stop, night_start)
        results_temp = cursor.execute(query).fetchall()
        results = []
        
        # apply additional conditions:
        for __ in range(len(results_temp)):
            keep = True
            result = results_temp.pop(0)
            
            # condition: telescope:
            if telescope is None:
                pass
            elif telescope != result['telescope']:
                keep = False
            
            # condition: pending or not:
            if pending is None:
                pass
            elif pending and result['nobs_pending'] == 0:
                keep = False
            elif not pending and result['nobs_pending'] > 0:
                keep = False
            
            # condition: active or not:
            if active is None:
                pass
            elif active and not result['active']:
                keep = False
            elif not active and result['active']:
                keep = False
            
            if keep:
                results.append(result)
            
    return results

In [79]:
DataFrame(query())

,field_id,fov,center_ra,center_dec,tilt,telescope,active,jd_first,jd_next,nobs_tot,nobs_done,nobs_pending,date_start,date_stop,duration,status,setting_duration
0,2094,0.008727,3.653620,0.021817,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1,2024-01-16 01:06:00.000,2024-01-16 04:29:36.381,0.141393,rising,None
1,2095,0.008727,3.662299,0.021817,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1,2024-01-16 01:08:00.000,2024-01-16 04:29:36.381,0.140004,rising,None
2,2096,0.008727,3.670977,0.021817,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1,2024-01-16 01:10:00.000,2024-01-16 04:29:36.381,0.138616,rising,None
3,2097,0.008727,3.679655,0.021817,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1,2024-01-16 01:12:00.000,2024-01-16 04:29:36.381,0.137227,rising,None
4,2098,0.008727,3.688334,0.021817,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1,2024-01-16 01:14:00.000,2024-01-16 04:29:36.381,0.135838,rising,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63317,81623,0.008727,2.219604,1.317724,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1,2024-01-15 17:51:00.000,2024-01-16 04:28:00.000,0.442361,rising,None
63318,81624,0.008727,2.253751,1.317724,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1,2024-01-15 17:59:00.000,2024-01-16 04:29:36.381,0.437921,rising,None
63319,81625,0.008727,2.287899,1.317724,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1,2024-01-15 18:07:00.000,2024-01-16 04:29:36.381,0.432366,rising,None
63320,81626,0.008727,2.322047,1.317724,0.0,Skinakas,1,2460310.5,2460370.5,1,0,1,2024-01-15 18:14:00.000,2024-01-16 04:29:36.381,0.427504,rising,None


In [80]:
rep = 10
runtime = timeit.timeit(query, number=rep) / rep
print(f'Runtime: {runtime:.3f} s')

Runtime: 3.226 s


**Result:** 
1. This python post-filtering approach is faster than the SQL query with all conditions. Note that we are comparing fetching all rows in the last test and fetching only the first row in the plain-SQL test.
2. The post-filtering only increases the runtime by around 1 second, relative to the test case further above that did not apply the "pending" and "active" conditions.
3. This is a very feasible approach.